# **calt Demo Notebook**

This notebook shows a minimal end‑to‑end workflow for the **calt** library:

1. **Install and import** the library  
2. **Generate** a dataset of *polynomial‑sum* examples  
3. **Configure** the tokenizer and model  
4. **Train** the Transformer  
5. **Visualize** training result  


Note on Google Colab: 
- Change the runtime type to GPU (e.g., T4 GPU) from the Runtime tab -> Change runtime type -> GPU
- The `Sympy` backend to simplify the installation dependencies. For extensive usage, we recommend using the `SageMath` backend, which for example allows parallel sample generations.     

## 1  – Installation & Imports  
Run the next cell to ensure **calt** and its dependencies are installed, then import the required Python packages.  


In [3]:
%%capture
!pip install calt-x

In [3]:
from typing import List, Tuple
import random
from sympy import GF, ZZ
from sympy.polys.rings import ring, PolyElement
from transformers import BartConfig, BartForConditionalGeneration as Transformer
from transformers import TrainingArguments
from calt import (
    PolynomialTrainer,
    data_loader,
)
from calt.generator.sympy import (
    PolynomialSampler,
    DatasetGenerator,
    DatasetWriter,
)
from calt.data_loader.utils import (
    load_eval_results,
    parse_poly,
    display_with_diff
)
import torch, random, numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

ImportError: cannot import name 'PolynomialSampler' from 'calt.generator.sympy' (/home/ara_shun/workspace/calt/src/calt/generator/sympy/__init__.py)

## 2  – Dataset Generation *(Polynomial Addition)*  
This cell uses `calt.generator` utilities to create a synthetic dataset of polynomial‑addition.

In [4]:
class SumProblemGenerator:
    """
    Problem generator for polynomial sum problems.

    This generator creates problems in which the input is a list of polynomials F = [f_1, f_2, ..., f_n],
    and the output is a single polynomial g = f_1 + f_2 + ... + f_n.
    """

    def __init__(
        self, sampler: PolynomialSampler, max_polynomials: int, min_polynomials: int
    ):
        """
        Initialize polynomial sum generator.

        Args:
            sampler: Polynomial sampler
            max_polynomials: Maximum number of polynomials in F
            min_polynomials: Minimum number of polynomials in F
        """
        self.sampler = sampler
        self.max_polynomials = max_polynomials
        self.min_polynomials = min_polynomials

    def __call__(self, seed: int) -> Tuple[List[PolyElement], PolyElement]:
        """
        Generate a single sample.

        Each sample consists of:
        - Input polynomial system F
        - Output polynomial g (sum of F)

        Args:
            seed: Seed for random number generator
        """
        random.seed(seed)

        # Choose number of polynomials for this sample
        num_polys = random.randint(self.min_polynomials, self.max_polynomials)

        # Generate input polynomials using sampler
        F = self.sampler.sample(num_samples=num_polys)

        # Generate output polynomial g (sum of F)
        g = sum(F)

        return F, g


In [15]:
save_dir = "."

# set up polynomial ring
R, *gens = ring("x0,x1", GF(7), order="grevlex")
# Initialize polynomial sampler
sampler = PolynomialSampler(
    ring=R,
    max_num_terms=2,
    max_degree=2,
    min_degree=1,
    degree_sampling="uniform",  # "uniform" or "fixed"
    term_sampling="uniform",  # "uniform" or "fixed"
    max_coeff=None,  # Used for RR and ZZ
    num_bound=None,  # Used for QQ
    strictly_conditioned=False,
    nonzero_instance=True,
)
# Initialize problem generator
problem_generator = SumProblemGenerator(
    sampler=sampler,
    max_polynomials=2,
    min_polynomials=2,
)
# Initialize dataset generator
dataset_generator = DatasetGenerator(
    backend="multiprocessing",
    n_jobs=1,  # warning: the current version with Sympy backend only supports n_jobs=1.
    verbose=True,
    root_seed=100,
)
# Generate training set
train_samples, _ = dataset_generator.run(
    train=True,
    num_samples=10000,
    problem_generator=problem_generator,
)
# Generate test set
test_samples, _ = dataset_generator.run(
    train=False,
    num_samples=1000,
    problem_generator=problem_generator,
)
# Initialize writer
dataset_writer = DatasetWriter(save_dir)
# Save datasets
dataset_writer.save_dataset(train_samples, tag="train")
dataset_writer.save_dataset(test_samples, tag="test")

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 1249 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 2449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 3199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 4999 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 6049 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 7199 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 8449 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 9799 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[

## 3  – Model Configuration  
Here we instantiate the tokenizer, define the Transformer architecture, and prepare the training pipeline.  


In [16]:
# Point to any dataset you like; here we assume the toy Sum dataset from the data‑generation notebook.
TRAIN_PATH = "train_raw.txt"
TEST_PATH = "test_raw.txt"
dataset, tokenizer, data_collator = data_loader(
    train_dataset_path=TRAIN_PATH,
    test_dataset_path=TEST_PATH,
    field="GF7",
    num_variables=2,
    max_degree=10,  # Should cover the range of generated samples
    max_coeff=7,   # Should cover the range of generated samples
    max_length=256,
)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [17]:
# Minimal architecture.
model_cfg = BartConfig(
    d_model=256,       # 'width' of the model
    vocab_size=len(tokenizer.vocab),
    encoder_layers=2,  # 'depth' of encoder network
    decoder_layers=2,  # 'depth' of decoder network
    max_position_embeddings=256,  # max length of input/output
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    decoder_start_token_id=tokenizer.bos_token_id,
    max_length=256,  # max length of input/output
)
model = Transformer(config=model_cfg)

## 4  – Training Hyper‑parameters  
Learning‑rate schedule, batch size, number of epochs, and other trainer options are declared in this cell.  


In [18]:
args = TrainingArguments(
    output_dir="results/",
    num_train_epochs=20,
    logging_steps=50,
    per_device_train_batch_size=int(128),
    per_device_eval_batch_size=int(128),
    save_strategy="no",  # skip checkpoints for the quick demo
    seed=SEED,
    remove_unused_columns=False,
    label_names=["labels"],
    report_to="none",
)

## 5  – Model Training  
Launch the training loop. Progress is typically logged to the console (and optionally to Weights & Biases).  


In [ ]:
trainer = PolynomialTrainer(
    args=args,
    model=model,
    processing_class=tokenizer,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# train
results = trainer.train()
trainer.save_model()
metrics = results.metrics

# eval
eval_metrics = trainer.evaluate()
metrics.update(eval_metrics)
success_rate = trainer.generate_evaluation(max_length=128)
metrics["success_rate"] = success_rate

# save metrics
trainer.save_metrics("all", metrics)

print(f'success rate on test set: {100*metrics["success_rate"]:.1f} %')

## 6  – Visualizing Training Results  
Finally, we visualize the differences between the mispredicted samples and their correct counterparts. 


In [ ]:
gen_texts, ref_texts = load_eval_results("results/eval_results.json")

success_cases = [(i, gen, ref) for i, (gen, ref) in enumerate(zip(gen_texts, ref_texts)) if gen == ref]
failure_cases = [(i, gen, ref) for i, (gen, ref) in enumerate(zip(gen_texts, ref_texts)) if gen != ref]

num_show = 5

print('-------------------------')
print(''' Success cases ''')
print('-------------------------')
for (i, gen, ref) in success_cases[:num_show]:
    gen_expr = parse_poly(gen, ["x", "y"])
    ref_expr = parse_poly(ref, ["x", "y"])

    print(f"===== sample id: {i+1} =====")
    display_with_diff(ref_expr, gen_expr)



print('\n-------------------------')
print(''' Failure cases ''')
print('-------------------------')
for (i, gen, ref) in failure_cases[:num_show]:
    gen_expr = parse_poly(gen, ["x", "y"])
    ref_expr = parse_poly(ref, ["x", "y"])

    print(f"===== sample id: {i+1} =====")
    display_with_diff(ref_expr, gen_expr)
